In [1]:
import sys
sys.path.insert(0, "../../../")

In [2]:
import deeptorch as dtm

import torch
import torch.nn as nn
import torchvision
import pytorch_lightning as pl

In [3]:
# Load the MNIST dataset
mnist = torchvision.datasets.MNIST(
    root="data", train=True, download=True, transform=torchvision.transforms.ToTensor()
)

mnist_test = torchvision.datasets.MNIST(
    root="data", train=False, download=True, transform=torchvision.transforms.ToTensor()
)

In [4]:
mnist_dataloader = torch.utils.data.DataLoader(mnist, batch_size=32, num_workers=4)
mnist_test_dataloader = torch.utils.data.DataLoader(mnist_test, batch_size=32, num_workers=4)

#### 1.1 Default ImageClassifier

In [5]:
classifier = dtm.ImageClassifier(num_classes=10)
print(classifier)
# Not necessary, but to get the number of parameters we first need to run the model with a batch of data
# This is because the model is initialized lazily
classifier(torch.rand(1, 1, 28, 28))

trainer = pl.Trainer(max_epochs=5, accelerator="cuda")
trainer.fit(classifier, mnist_dataloader, mnist_test_dataloader)
trainer.test(dataloaders=mnist_test_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


ImageClassifier(
  (backbone): ConvolutionalEncoder(
    (blocks): ModuleList(
      (0): Template(
        (layer): LazyConv2d(0, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activation): ReLU()
        (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (1): Template(
        (layer): LazyConv2d(0, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activation): ReLU()
        (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (2): Template(
        (layer): LazyConv2d(0, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activation): ReLU()
        (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (3): Template(
        (layer): LazyConv2d(0, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activation): ReLU()
        (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mo

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type                          | Params
---------------------------------------------------------------
0 | backbone     | ConvolutionalEncoder          | 24.4 K
1 | connector    | Flatten                       | 0     
2 | head         | CategoricalClassificationHead | 650   
3 | val_accuracy | Accuracy                      | 0     
---------------------------------------------------------------
25.0 K    Trainable params
0         Non-trainable params
25.0 K    Total params
0.100     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.
c:\Users\GU\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\checkpoint_connector.py:134: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(
Restoring states from the checkpoint path at c:\Users\GU\DeepTorch\examples\vision\classification\lightning_logs\version_17\checkpoints\epoch=4-step=9375.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at c:\Users\GU\DeepTorch\examples\vision\classification\lightning_logs\version_17\checkpoints\epoch=4-step=9375.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9853000044822693     │
│         test_loss         │     1.476316213607788     │
└───────────────────────────┴───────────────────────────┘

[{'test_acc': 0.9853000044822693, 'test_loss': 1.476316213607788}]

# 1.2 With LeakyReLU

In [6]:
classifier = dtm.ImageClassifier(
    num_classes=10,
    backbone=dtm.Config()
                .__.activation(nn.LeakyReLU, negative_slope=0.1)
)
print(classifier)

# Not necessary, but to get the number of parameters we first need to run the model with a batch of data
# This is because the model is initialized lazily
classifier(torch.rand(1, 1, 28, 28))

trainer = pl.Trainer(max_epochs=5, accelerator="cuda")
trainer.fit(classifier, mnist_dataloader, mnist_test_dataloader)
trainer.test(dataloaders=mnist_test_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type                          | Params
---------------------------------------------------------------
0 | backbone     | ConvolutionalEncoder          | 24.4 K
1 | connector    | Flatten                       | 0     
2 | head         | CategoricalClassificationHead | 650   
3 | val_accuracy | Accuracy                      | 0     
---------------------------------------------------------------
25.0 K    Trainable params
0         Non-trainable params
25.0 K    Total params
0.100     Total estimated model params size (MB)


ImageClassifier(
  (backbone): ConvolutionalEncoder(
    (blocks): ModuleList(
      (0): Template(
        (layer): LazyConv2d(0, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activation): LeakyReLU(negative_slope=0.1)
        (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (1): Template(
        (layer): LazyConv2d(0, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activation): LeakyReLU(negative_slope=0.1)
        (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (2): Template(
        (layer): LazyConv2d(0, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activation): LeakyReLU(negative_slope=0.1)
        (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (3): Template(
        (layer): LazyConv2d(0, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activation): LeakyReLU(negative_

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.
Restoring states from the checkpoint path at c:\Users\GU\DeepTorch\examples\vision\classification\lightning_logs\version_18\checkpoints\epoch=4-step=9375.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at c:\Users\GU\DeepTorch\examples\vision\classification\lightning_logs\version_18\checkpoints\epoch=4-step=9375.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9812999963760376     │
│         test_loss         │     1.479689598083496     │
└───────────────────────────┴───────────────────────────┘

[{'test_acc': 0.9812999963760376, 'test_loss': 1.479689598083496}]

# 1.3 With BatchNorm2d

In [7]:
block = dtm.Node("layer") >> dtm.Node("activation") >> dtm.Node("normalization") >> dtm.Node("pool")

classifier = dtm.ImageClassifier(
    num_classes=10,
    backbone=dtm.Config()
                .blocks(block)
                .blocks.activation(nn.LeakyReLU, negative_slope=0.1)
                .blocks.normalization(nn.LazyBatchNorm2d)
                
)

# Not necessary, but to get the number of parameters we first need to run the model with a batch of data
# This is because the model is initialized lazily
classifier(torch.rand(1, 1, 28, 28))

trainer = pl.Trainer(max_epochs=5, accelerator="cuda")
trainer.fit(classifier, mnist_dataloader, mnist_test_dataloader)
trainer.test(dataloaders=mnist_test_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type                          | Params
---------------------------------------------------------------
0 | backbone     | ConvolutionalEncoder          | 24.6 K
1 | connector    | Flatten                       | 0     
2 | head         | CategoricalClassificationHead | 650   
3 | val_accuracy | Accuracy                      | 0     
---------------------------------------------------------------
25.3 K    Trainable params
0         Non-trainable params
25.3 K    Total params
0.101     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.
Restoring states from the checkpoint path at c:\Users\GU\DeepTorch\examples\vision\classification\lightning_logs\version_19\checkpoints\epoch=4-step=9375.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at c:\Users\GU\DeepTorch\examples\vision\classification\lightning_logs\version_19\checkpoints\epoch=4-step=9375.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9854999780654907     │
│         test_loss         │    1.4761897325515747     │
└───────────────────────────┴───────────────────────────┘

[{'test_acc': 0.9854999780654907, 'test_loss': 1.4761897325515747}]

# 1.4 Larger head

In [8]:
block = dtm.Node("layer") >> dtm.Node("activation") >> dtm.Node("normalization") >> dtm.Node("pool")

# out_layer_act = 
output = dtm.Node("extra_layer") >> dtm.Node("extra_activation") >> dtm.Node("layer") >> dtm.Node("activation")

classifier = dtm.ImageClassifier(
    num_classes=10,
    backbone=dtm.Config()
                .blocks(block)
                .blocks.activation(nn.LeakyReLU, negative_slope=0.1)
                .blocks.normalization(nn.LazyBatchNorm2d),
    head=dtm.Config()
            .output(output)
            .output.extra_layer(nn.LazyLinear, out_features=128)
            .output.extra_activation(nn.LeakyReLU, negative_slope=0.1)
)
print(classifier)

# # Not necessary, but to get the number of parameters we first need to run the model with a batch of data
# # This is because the model is initialized lazily
classifier(torch.rand(1, 1, 28, 28))

trainer = pl.Trainer(max_epochs=5, accelerator="cuda")
trainer.fit(classifier, mnist_dataloader, mnist_test_dataloader)
trainer.test(dataloaders=mnist_test_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type                          | Params
---------------------------------------------------------------
0 | backbone     | ConvolutionalEncoder          | 24.6 K
1 | connector    | Flatten                       | 0     
2 | head         | CategoricalClassificationHead | 9.6 K 
3 | val_accuracy | Accuracy                      | 0     
---------------------------------------------------------------
34.2 K    Trainable params
0         Non-trainable params
34.2 K    Total params
0.137     Total estimated model params size (MB)


ImageClassifier(
  (backbone): ConvolutionalEncoder(
    (blocks): ModuleList(
      (0): Template(
        (layer): LazyConv2d(0, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activation): LeakyReLU(negative_slope=0.1)
        (normalization): LazyBatchNorm2d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (1): Template(
        (layer): LazyConv2d(0, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activation): LeakyReLU(negative_slope=0.1)
        (normalization): LazyBatchNorm2d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (2): Template(
        (layer): LazyConv2d(0, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activation): LeakyReLU(negative_slope=0.1)
        (normalization): LazyBa

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.
Restoring states from the checkpoint path at c:\Users\GU\DeepTorch\examples\vision\classification\lightning_logs\version_20\checkpoints\epoch=4-step=9375.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at c:\Users\GU\DeepTorch\examples\vision\classification\lightning_logs\version_20\checkpoints\epoch=4-step=9375.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9851999878883362     │
│         test_loss         │    1.4760082960128784     │
└───────────────────────────┴───────────────────────────┘

[{'test_acc': 0.9851999878883362, 'test_loss': 1.4760082960128784}]